Description des entreprises qui divulguent / ne divulguent pas leurs données de gender pay gap

In [1]:
!pip install openpyxl


In [2]:
import pandas as pd
import numpy as np
import math
import openpyxl


In [3]:
file_imp_path = "data/ESG_raw_data_07_02.xlsx"
file_sect = "data/secteurs.csv"
corresp_secteurs = pd.read_csv(file_sect, sep=";")
imp_data = pd.read_excel(file_imp_path)
imp_data = imp_data.join(corresp_secteurs.set_index('primary_industry'), on = "primary_industry")

# Premières stats desc

In [4]:
data_size = imp_data.shape

print("Size of the DataFrame:")
print("Nombre d'observations:", data_size[0])
print("Nombre d'entreprises", imp_data['company_id'].nunique())
print("Nombre moyen d'employés :", imp_data['employees'].mean())

variable_list = imp_data.columns.tolist()
# Display the list of variables
print("List of Variables:")
print(variable_list)

Size of the DataFrame:
Nombre d'observations: 40554
Nombre d'entreprises 13518
Nombre moyen d'employés : 13142.447727480847
List of Variables:
['company_id', 'year', 'company_name', 'ticker', 'LEI', 'isin', 'Business Desc.', 'region', 'hq_country', 'primary_industry', 'market_cap', 'employees', 'revenue', 'scope_1', 'scope_2', 'scope_3', 'waste_production', 'waste_recycling', 'water_consumption', 'water_withdrawal', 'energy_consumption', 'hours_of_training', 'independent_board_members_percentage', 'legal_costs_paid_for_controversies', 'ceo_compensation', 'gender_pay_gap', 'secteur', 'sous_secteur']


In [5]:
statistics_employees = imp_data['employees'].describe()

# Extraire les quartiles et la médiane
quartiles = [statistics_employees['25%'], statistics_employees['50%'], statistics_employees['75%']]
median = statistics_employees['50%']

# Afficher les quartiles et la médiane
print("Quartiles de la base imp_data['employees'] :")
print("Premier quartile (Q1) :", quartiles[0])
print("Médiane (Q2) :", median)
print("Troisième quartile (Q3) :", quartiles[2])

# Trouver les valeurs extrêmes de la colonne "employees"
min_employees = imp_data['employees'].min()
max_employees = imp_data['employees'].max()
print("Valeur minimale de la base imp_data['employees'] :", min_employees)
print("Valeur maximale de la base imp_data['employees'] :", max_employees)

quantile_99 = imp_data['employees'].quantile(0.99)
print("Quantile 99 de la base imp_data['employees'] :", quantile_99)

quantile_05 = imp_data['employees'].quantile(0.05)
print("Quantile 01 de la base imp_data['employees'] :", quantile_05)


Quartiles de la base imp_data['employees'] :
Premier quartile (Q1) : 1035.0
Médiane (Q2) : 3049.0
Troisième quartile (Q3) : 9362.0
Valeur minimale de la base imp_data['employees'] : 1.0
Valeur maximale de la base imp_data['employees'] : 2300000.0
Quantile 99 de la base imp_data['employees'] : 184575.20000000022
Quantile 01 de la base imp_data['employees'] : 192.0


In [6]:
# Create a table with the % of observations per region 

region_counts = imp_data['region'].value_counts(normalize=True) * 100

# Create a DataFrame from the value counts
region_percentage = pd.DataFrame({'Region': region_counts.index, '% of Observations': region_counts.values})

# Display the DataFrame
print(region_percentage)

                        Region  % of Observations
0               Asia / Pacific          59.890516
1     United States and Canada          19.751443
2                       Europe          13.729842
3         Africa / Middle East           3.491641
4  Latin America and Caribbean           3.136559


In [7]:
# Create a table with the % of observations per country 

region_counts = imp_data['hq_country'].value_counts(normalize=True) * 100

# Create a DataFrame from the value counts
region_percentage = pd.DataFrame({'Pays': region_counts.index, '% of Observations': region_counts.values})

# Display the DataFrame
print(region_percentage)

                       Pays  % of Observations
0                     China          31.143660
1             United States          17.465601
2                     Japan           7.767421
3                     India           4.793609
4                    Taiwan           3.388075
..                      ...                ...
97                  Bahamas           0.007398
98                 Bulgaria           0.007398
99                Gibraltar           0.007398
100  British Virgin Islands           0.007398
101                 Ukraine           0.007398

[102 rows x 2 columns]


In [8]:
# Create a table with the % of observations per sector 

region_counts = imp_data['secteur'].value_counts(normalize=True) * 100

# Create a DataFrame from the value counts
region_percentage = pd.DataFrame({'Secteur': region_counts.index, '% of Observations': region_counts.values})

# Display the DataFrame
print(region_percentage)

                                       Secteur  % of Observations
0                   Industrials, capital goods          13.678059
1                       Information Technology          12.856931
2                                    Materials          11.577156
3                       Consumer Discretionary          10.948365
4                                   Financials          10.933570
5                                  Health Care           9.720373
6                             Consumer Staples           8.477585
7   Industrials, commercial and transportation           6.561622
8                       Communication Services           4.438526
9                                  Real Estate           3.935493
10                                      Energy           3.484243
11                                   Utilities           3.388075


In [9]:
materials_data = imp_data[imp_data['secteur'] == 'Materials']

# Compter le nombre d'entreprises pour chaque industrie principale
industry_counts = materials_data['primary_industry'].value_counts()

# Afficher les résultats
print("Primary Industry Counts in the Materials Sector:")
print(industry_counts)

Primary Industry Counts in the Materials Sector:
primary_industry
Commodity Chemicals                       1068
Specialty Chemicals                        717
Steel                                      603
Construction Materials                     420
Diversified Metals and Mining              384
Fertilizers and Agricultural Chemicals     345
Gold                                       207
Aluminum                                   171
Paper Products                             156
Metal and Glass Containers                 132
Paper Packaging                            129
Diversified Chemicals                      117
Copper                                      75
Forest Products                             66
Industrial Gases                            42
Precious Metals and Minerals                39
Silver                                      24
Name: count, dtype: int64


In [10]:
imp_data['missing'] = imp_data['gender_pay_gap'].isnull().astype(float)

missing_paygap_subdataset = imp_data[imp_data['missing'] == 1]
non_missing_paygap_subdataset = imp_data[imp_data['missing'] == 0]


data_size = non_missing_paygap_subdataset.shape
#405 observations dans la base non_missing sur 40K 
print("Size of the DataFrame:")
print("Number of rows:", data_size[0])
print("Number of columns:", data_size[1])


Size of the DataFrame:
Number of rows: 405
Number of columns: 29


# Niveau moyen de l'ecart de salaire horaire hommes-femmes dans les entreprise qui divulgent

In [11]:
import scipy.stats as stats

gender_pay_gap_without_missing = non_missing_paygap_subdataset['gender_pay_gap']
sample_mean = gender_pay_gap_without_missing.mean()
standard_error = stats.sem(gender_pay_gap_without_missing)
margin_of_error = standard_error * stats.t.ppf(0.975, len(gender_pay_gap_without_missing) - 1)
confidence_interval = (sample_mean - margin_of_error, sample_mean + margin_of_error)
print("Niveau moyen", sample_mean)
print("95% Confidence Interval of Average Gender Pay Gap (excluding missing values):", confidence_interval)

Niveau moyen 11.86492098765432
95% Confidence Interval of Average Gender Pay Gap (excluding missing values): (10.738791602639328, 12.991050372669314)


# Identification des variables qui influencent fortement la divulgation

In [12]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

imp_data['region_corrected'] = imp_data['region']
imp_data.loc[imp_data['hq_country'] == 'China', 'region_corrected'] = 'China'
imp_data.loc[imp_data['region_corrected'] == 'Asia / Pacific', 'region_corrected'] = 'Asia / Pacific without China'


imp_data['waste_production_per_employee'] = imp_data['waste_production'] / imp_data['employees']
imp_data['waste_recycling_per_employee'] = imp_data['waste_recycling'] / imp_data['employees']
imp_data['scope_1_per_employee'] = imp_data['scope_1'] / imp_data['employees']
imp_data['scope_2_per_employee'] = imp_data['scope_2'] / imp_data['employees']
imp_data['scope_3_per_employee'] = imp_data['scope_3'] / imp_data['employees']
imp_data['water_consumption_per_employee'] = imp_data['water_consumption'] / imp_data['employees']
imp_data['water_withdrawal_per_employee'] = imp_data['water_withdrawal'] / imp_data['employees']
imp_data['energy_consumption_per_employee'] = imp_data['energy_consumption'] / imp_data['employees']
imp_data['hours_of_training_per_employee'] = imp_data['hours_of_training'] / imp_data['employees']
imp_data['legal_costs_paid_for_controversies_per_employee'] = imp_data['legal_costs_paid_for_controversies'] / imp_data['employees']



X = imp_data[['secteur', 'region_corrected', 'hq_country', 'employees',
 'revenue', 'scope_1_per_employee', 'scope_2_per_employee', 'scope_3_per_employee', 'waste_production_per_employee', 
 'waste_recycling_per_employee', 'water_consumption_per_employee', 'water_withdrawal_per_employee', 
              'energy_consumption_per_employee', 'hours_of_training_per_employee', 
              'independent_board_members_percentage', 'legal_costs_paid_for_controversies',
 'ceo_compensation', 'missing']]


X = pd.get_dummies(X, columns=['region_corrected', 'hq_country', 'secteur'], drop_first=True)

# Ensuring that categorical variables are encoded as integers
region_columns = [col for col in X.columns if col.startswith('region_corrected') or col.startswith('secteur')
                 or col.startswith('hq_country')]
X[region_columns] = X[region_columns].astype(float)

X = X.astype(float)

# Apply scaling to the entire DataFrame
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_transformed = pd.DataFrame(X_scaled, columns=X.columns)


# Compute the covariance matrix
corr_matrix = X_transformed.corr()

correlation_with_missing = corr_matrix['missing'].abs().sort_values(ascending=False)

# Select the top 10 correlated variables with highest absolute correlation
top_10_correlated = correlation_with_missing[1:11]  # Exclude 'missing' itself

# Create a DataFrame to hold the top 10 correlated variables with both absolute and non-absolute correlation values
top_10_correlated_df = pd.DataFrame({
    'Variable': top_10_correlated.index,
    'Correlation with missing': corr_matrix['missing'][top_10_correlated.index].values,
    'Absolute Correlation with missing': top_10_correlated.values
})

# Print the DataFrame
print("Top 10 Correlated Variables with highest absolute correlation with 'missing':")
print(top_10_correlated_df)

# Explication taille + localisation Europe/asie sans secteur particulier semble apparaitre
# Plus de divulgation en Europe / - en Asie
# Plus de divulgation quand taille augmente
# Plus de divulgation associé également à water recycling

Top 10 Correlated Variables with highest absolute correlation with 'missing':
                                        Variable  Correlation with missing  \
0                      hq_country_United Kingdom                 -0.218913   
1                        region_corrected_Europe                 -0.193395   
2                               hq_country_Spain                 -0.102673   
3                         region_corrected_China                  0.066475   
4                               hq_country_China                  0.066475   
5                                        revenue                 -0.062693   
6                 water_consumption_per_employee                 -0.060108   
7                                      employees                 -0.049331   
8  region_corrected_Asia / Pacific without China                  0.048452   
9                              hq_country_France                 -0.045801   

   Absolute Correlation with missing  
0                       

# Regression de missing sur les variables determinantes

In [13]:
from scipy.stats.mstats import winsorize
import statsmodels.api as sm

# Filtre les données pour enlever les entreprises trop petites qui ont potentiellement des carac
# très dif des multinationales étudiées

X = imp_data[['revenue', 'employees', 'region_corrected','secteur','year']]

X = pd.get_dummies(X, columns=['region_corrected','secteur','year'])


# Convert variables starting with "region" to integers
region_columns = [col for col in X.columns if col.startswith('region_corrected') or col.startswith('secteur')
                 or col.startswith('year')]
X[region_columns] = X[region_columns].astype(int)

print(region_columns)

# Selectionne la catégorie de référence
X.drop(columns='region_corrected_United States and Canada', inplace=True)
X.drop(columns='secteur_Real Estate', inplace=True)
X.drop(columns='year_2018', inplace=True)


# Add constant to independent variables
X = sm.add_constant(X)
y = imp_data['missing']

y_filtered = y.dropna()
X_filtered = X.dropna()
common_index = y_filtered.index.intersection(X_filtered.index)
y = y_filtered.loc[common_index]
X = X_filtered.loc[common_index]
groups =  imp_data['year'].loc[common_index]

# Fit Logit model
model_logit = sm.Logit(y, X)
result = model_logit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())

# Fit Logit model
model_probit = sm.Probit(y, X)
result = model_probit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())


# On retrouve des effets marginaux moyens cohérents avec la LPM 
# Que les données soient filtrées ou pas -> meme resultats

correlation = imp_data['revenue'].corr(imp_data['employees'])
print("Correlation between revenue and employees:", correlation)


# Fit LPM
model = sm.OLS(y, X)
result = model.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(result.summary())

quantile_05 = imp_data['employees'].quantile(0.05)
filtered_data = imp_data[imp_data['employees'] > quantile_05]



X = imp_data[['revenue', 'employees', 'region_corrected','secteur','year']]





X = filtered_data[['revenue', 'employees', 'region_corrected','secteur','year']]

X = pd.get_dummies(X, columns=['region_corrected','secteur','year'])


# Convert variables starting with "region" to integers
region_columns = [col for col in X.columns if col.startswith('region_corrected') or col.startswith('secteur')
                 or col.startswith('year')]
X[region_columns] = X[region_columns].astype(int)

# Selectionne la catégorie de référence
X.drop(columns='region_corrected_United States and Canada', inplace=True)
X.drop(columns='secteur_Real Estate', inplace=True)
X.drop(columns='year_2018', inplace=True)



# Add constant to independent variables
X = sm.add_constant(X)
y = filtered_data['missing']
y_filtered = y.dropna()
X_filtered = X.dropna()
common_index = y_filtered.index.intersection(X_filtered.index)
y = y_filtered.loc[common_index]
X = X_filtered.loc[common_index]
groups =  filtered_data['year'].loc[common_index]

# Fit Logit model
model_logit = sm.Logit(y, X)
result = model_logit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())


['region_corrected_Africa / Middle East', 'region_corrected_Asia / Pacific without China', 'region_corrected_China', 'region_corrected_Europe', 'region_corrected_Latin America and Caribbean', 'region_corrected_United States and Canada', 'secteur_Communication Services', 'secteur_Consumer Discretionary', 'secteur_Consumer Staples', 'secteur_Energy', 'secteur_Financials', 'secteur_Health Care', 'secteur_Industrials, capital goods', 'secteur_Industrials, commercial and transportation', 'secteur_Information Technology', 'secteur_Materials', 'secteur_Real Estate', 'secteur_Utilities', 'year_2018', 'year_2019', 'year_2020']
Optimization terminated successfully.
         Current function value: 0.049610
         Iterations 13
        Logit Marginal Effects       
Dep. Variable:                missing
Method:                          dydx
At:                           overall
                                                        dy/dx    std err          z      P>|z|      [0.025      0.975]


/opt/mamba/lib/python3.12/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 20, but rank is 2
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:                missing   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     2.866
Date:                Mon, 06 May 2024   Prob (F-statistic):              0.259
Time:                        12:01:23   Log-Likelihood:                 26490.
No. Observations:               32695   AIC:                        -5.294e+04
Df Residuals:                   32674   BIC:                        -5.276e+04
Df Model:                          20                                         
Covariance Type:              cluster                                         
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------

# Lien entre missing et les autres variables sociales

In [17]:
# varia sociales potentiellement reliées : 
# Hours of training per employee 
# legal_costs_paid_for_controversies
# independent_board_members_percentage

X = imp_data[['hours_of_training_per_employee','independent_board_members_percentage',
              'legal_costs_paid_for_controversies_per_employee','employees','region_corrected','secteur','year']]


X = pd.get_dummies(X, columns=['region_corrected','secteur','year'])
# Convert variables starting with "region" to integers
region_columns = [col for col in X.columns if col.startswith('region_corrected') or col.startswith('secteur')
                 or col.startswith('year')]
X[region_columns] = X[region_columns].astype(int)

print(region_columns)

# Selectionne la catégorie de référence
X.drop(columns='region_corrected_United States and Canada', inplace=True)
X.drop(columns='secteur_Real Estate', inplace=True)
X.drop(columns='year_2018', inplace=True)



# Add constant to independent variables
X = sm.add_constant(X)
y = imp_data['missing']
y_filtered = y.dropna()
X_filtered = X.dropna()
common_index = y_filtered.index.intersection(X_filtered.index)
y = y_filtered.loc[common_index]
X = X_filtered.loc[common_index]
groups =  imp_data['year'].loc[common_index]


# Fit Logit model
model_logit = sm.Logit(y, X)
result = model_logit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())

# Fit probit model
model_probit = sm.Probit(y, X)
result = model_probit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())


# Fit LPM
model = sm.OLS(y, X,missing ="drop")
result = model.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(result.summary())

# Effets marginaux sont modifiés MAIS reste quand meme avec les memes conclusions -> ajout de ceo_compensation fait
# completement foirer la reg meme si je winsorise

# Pas de relation significative à 10% egalement avec le nombre d'heures de formations proposées 
# (différentes dimensiosn sociales)


# Winsorizing variables that have outliers
imp_data['winsorized_hours_of_training_per_employee'] = winsorize(imp_data['hours_of_training_per_employee'], limits=[0.05, 0.05])
imp_data["winsorized_legal_costs_paid_for_controversies_per_employee"] = winsorize(imp_data['legal_costs_paid_for_controversies_per_employee'], limits=[0.05, 0.05])
imp_data["winsorized_independent_board_members_percentage"] = winsorize(imp_data['independent_board_members_percentage'], limits=[0.05, 0.05])


X = imp_data[['winsorized_hours_of_training_per_employee','winsorized_independent_board_members_percentage',
              'winsorized_legal_costs_paid_for_controversies_per_employee','employees','region_corrected','secteur',
             'year']]



X = pd.get_dummies(X, columns=['region_corrected','secteur','year'])
# Convert variables starting with "region" to integers
region_columns = [col for col in X.columns if col.startswith('region_corrected') or col.startswith('secteur')
                 or col.startswith('year')]
X[region_columns] = X[region_columns].astype(int)

# Selectionne la catégorie de référence
X.drop(columns='region_corrected_United States and Canada', inplace=True)
X.drop(columns='secteur_Real Estate', inplace=True)
X.drop(columns='year_2018', inplace=True)


# Add constant to independent variables
X = sm.add_constant(X)
y = imp_data['missing']
y_filtered = y.dropna()
X_filtered = X.dropna()
common_index = y_filtered.index.intersection(X_filtered.index)
y = y_filtered.loc[common_index]
X = X_filtered.loc[common_index]
groups =  imp_data['year'].loc[common_index]

# Fit Logit model
model_logit = sm.Logit(y, X)
result = model_logit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())


['region_corrected_Africa / Middle East', 'region_corrected_Asia / Pacific without China', 'region_corrected_China', 'region_corrected_Europe', 'region_corrected_Latin America and Caribbean', 'region_corrected_United States and Canada', 'secteur_Communication Services', 'secteur_Consumer Discretionary', 'secteur_Consumer Staples', 'secteur_Energy', 'secteur_Financials', 'secteur_Health Care', 'secteur_Industrials, capital goods', 'secteur_Industrials, commercial and transportation', 'secteur_Information Technology', 'secteur_Materials', 'secteur_Real Estate', 'secteur_Utilities', 'year_2018', 'year_2019', 'year_2020']
Optimization terminated successfully.
         Current function value: 0.270672
         Iterations 10
        Logit Marginal Effects       
Dep. Variable:                missing
Method:                          dydx
At:                           overall
                                                        dy/dx    std err          z      P>|z|      [0.025      0.975]


/opt/mamba/lib/python3.12/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 22, but rank is 2
  warnings.warn('covariance of constraints does not have full '


        Logit Marginal Effects       
Dep. Variable:                missing
Method:                          dydx
At:                           overall
                                                                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------
winsorized_hours_of_training_per_employee                   3.228e-05   1.16e-05      2.773      0.006    9.46e-06    5.51e-05
winsorized_independent_board_members_percentage               -0.0629      0.019     -3.311      0.001      -0.100      -0.026
winsorized_legal_costs_paid_for_controversies_per_employee     0.0598      0.037      1.629      0.103      -0.012       0.132
employees                                                  -1.803e-08   3.91e-08     -0.461      0.645   -9.47e-08    5.87e-08
region_corrected_Africa / Middle East                         -0.0534      0.034     -

# Lien préoccupaton environnementales et préoccupations sociales

In [15]:
X = imp_data[[
              'scope_1_per_employee','scope_2_per_employee', 'scope_3_per_employee',
              'employees','region_corrected','secteur','year']]


X = pd.get_dummies(X, columns=['region_corrected','secteur','year'])
# Convert variables starting with "region" to integers
region_columns = [col for col in X.columns if col.startswith('region_corrected') or col.startswith('secteur')
                 or col.startswith('year')]
X[region_columns] = X[region_columns].astype(int)

# Selectionne la catégorie de référence
X.drop(columns='region_corrected_United States and Canada', inplace=True)
X.drop(columns='secteur_Real Estate', inplace=True)
X.drop(columns='year_2018', inplace=True)


# Add constant to independent variables
X = sm.add_constant(X)
y = imp_data['missing']
y_filtered = y.dropna()
X_filtered = X.dropna()
common_index = y_filtered.index.intersection(X_filtered.index)
y = y_filtered.loc[common_index]
X = X_filtered.loc[common_index]
groups =  imp_data['year'].loc[common_index]

# Fit Logit model
model_logit = sm.Logit(y, X)
result = model_logit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())

# Fit probit model
model_probit = sm.Probit(y, X)
result = model_probit.fit(cov_type='cluster', cov_kwds={'groups': groups})
margeff = result.get_margeff()
print(margeff.summary())

# Fit LPM
model = sm.OLS(y, X)
result = model.fit(cov_type='cluster', cov_kwds={'groups': groups})
print(result.summary())



# Ca converge pas si on inclut water_consumption / waste_recycling.. surement à cause du nombre trop faible
# d'observations dans ce cas du aux données manquantes

Optimization terminated successfully.
         Current function value: 0.112823
         Iterations 11
        Logit Marginal Effects       
Dep. Variable:                missing
Method:                          dydx
At:                           overall
                                                        dy/dx    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
scope_1_per_employee                                3.303e-06   3.04e-06      1.088      0.277   -2.65e-06    9.25e-06
scope_2_per_employee                                1.441e-05   1.12e-05      1.281      0.200   -7.63e-06    3.64e-05
scope_3_per_employee                                1.482e-07    2.2e-08      6.738      0.000    1.05e-07    1.91e-07
employees                                          -4.478e-08   8.39e-09     -5.335      0.000   -6.12e-08   -2.83e-08
region_corrected_Africa / Middl

/opt/mamba/lib/python3.12/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 22, but rank is 2
  warnings.warn('covariance of constraints does not have full '


# Stats desc sur données avant winso/ trimming

In [16]:
variables = ['revenue', 'employees', 'hours_of_training', 'legal_costs_paid_for_controversies', 'independent_board_members_percentage']

# Calculate statistics for each variable
stat_desc = pd.DataFrame(index=['min', 'max', 'mean', 'median', 'percentile_5', 'percentile_95'])
for variable in variables:
    stats = imp_data[variable].describe()
    percentile_95 = imp_data[variable].quantile(0.95)
    percentile_5 = imp_data[variable].quantile(0.05)
    median = imp_data[variable].median()  # Calculate median separately
    stats['percentile_5'] = percentile_5  # Assign top_5 value first
    stats['percentile_95'] = percentile_95  # Assign top_95 value second
    stats['median'] = median  # Assign median value
    stat_desc[variable] = stats

# Transpose the DataFrame for better readability
stat_desc = stat_desc.T

# Print the statistical description table
print(stat_desc)

                                           min           max           mean  \
revenue                              -13944.09  4.609835e+05    3371.594815   
employees                                 1.00  2.300000e+06   13142.447727   
hours_of_training                         0.00  1.482950e+08  983245.751363   
legal_costs_paid_for_controversies        0.00  1.483020e+04      34.627127   
independent_board_members_percentage      0.00  1.000000e+00       0.606068   

                                          median  percentile_5  percentile_95  
revenue                                  571.530     38.615000      13405.210  
employees                               3049.000    192.000000      53000.000  
hours_of_training                     144885.500     15.000000    4106365.700  
legal_costs_paid_for_controversies         0.000      0.000000         84.454  
independent_board_members_percentage       0.625      0.084847          1.000  
